# Быстрый старт - Загрузка данных Divvy Bikes

Этот notebook показывает как быстро начать работу с данными.

In [17]:
# Добавляем корень проекта в путь
import sys
from pathlib import Path

# Находим корень проекта (на уровень выше notebooks/)
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"Корень проекта: {project_root}")

Корень проекта: /Users/vsevolod/Desktop/divvy-bikes-analysis


In [18]:
# Импортируем необходимые библиотеки
import polars as pl
from src.data.load_data import load_raw_data, DataLoader

print("Импорты успешны")

Импорты успешны


In [19]:
   from src.data.load_data import load_raw_data
   trips = load_raw_data(year=2024)

FileNotFoundError: Не найдены файлы поездок за 2024 год

## 1. Загрузка данных за один год

In [20]:
# Загружаем данные за 2024 год
print("Загрузка данных за 2024 год...")
trips_2024 = load_raw_data(year=2024)

print(f"\nЗагружено поездок: {trips_2024.shape[0]:,}")
print(f"Колонок: {trips_2024.shape[1]}")
print(f"\nПервые 5 строк:")
trips_2024.head()

Загрузка данных за 2024 год...


FileNotFoundError: Не найдены файлы поездок за 2024 год

In [21]:
# Посмотрим на колонки
print("Колонки в датасете:")
for col in trips_2024.columns:
    print(f"  - {col}")

Колонки в датасете:


NameError: name 'trips_2024' is not defined

## 2. Базовая статистика

In [22]:
# Описательная статистика
trips_2024.describe()

NameError: name 'trips_2024' is not defined

## 3. Загрузка данных за несколько лет

In [23]:
# Используем DataLoader для загрузки диапазона лет
loader = DataLoader()

print("Загрузка данных за 2022-2024...")
trips_range = loader.load_raw_trips_range(2022, 2024)

print(f"\nЗагружено поездок: {trips_range.shape[0]:,}")

# Распределение по годам
yearly_stats = trips_range.group_by('year').agg(
    pl.count().alias('total_trips')
).sort('year')

print("\nПоездок по годам:")
yearly_stats

Пропуск 2022 года: данные не найдены
Пропуск 2023 года: данные не найдены
Пропуск 2024 года: данные не найдены


Загрузка данных за 2022-2024...

Загружено поездок: 0


/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_3101/2794487638.py:11: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('total_trips')


ColumnNotFoundError: unable to find column "year"; valid columns: []

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
DF []; PROJECT */0 COLUMNS

## 4. Простой анализ

In [24]:
# Пример анализа: топ-10 популярных станций отправления
# (Название колонки может отличаться в зависимости от года)

# Попробуем найти колонку со станцией отправления
station_cols = [col for col in trips_2024.columns if 'from' in col.lower() or 'start' in col.lower()]
print(f"Найденные колонки станций: {station_cols}")

if station_cols:
    station_col = station_cols[0]
    top_stations = (
        trips_2024
        .group_by(station_col)
        .agg(pl.count().alias('trips'))
        .sort('trips', descending=True)
        .head(10)
    )
    print(f"\nТоп-10 станций отправления:")
    display(top_stations)
else:
    print("Колонка со станцией не найдена")

NameError: name 'trips_2024' is not defined

## 5. Сохранение обработанных данных

In [25]:
# Пример: сохраним данные за 2024 в Parquet формат
from src.config.paths import INTERIM_DATA_DIR

output_path = INTERIM_DATA_DIR / 'trips_2024_raw.parquet'

loader.save_dataframe(
    trips_2024,
    output_path,
    format='parquet'
)

print(f"Данные сохранены: {output_path}")
print(f"Размер файла: {output_path.stat().st_size / 1024 / 1024:.1f} MB")

NameError: name 'trips_2024' is not defined

## 6. Проверка сохраненных данных

In [26]:
# Загрузим обратно для проверки
trips_loaded = pl.read_parquet(output_path)

print(f"Загружено из Parquet: {trips_loaded.shape[0]:,} строк")
print(f"Данные идентичны: {trips_2024.shape == trips_loaded.shape}")

FileNotFoundError: No such file or directory (os error 2): /Users/vsevolod/Desktop/divvy-bikes-analysis/data/interim/trips_2024_raw.parquet

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'sink'
